### Concatenation and Splitting Functions

In [7]:
import numpy as np

# Concatenate columns of an array
def concat_cols(A, B):
    return np.concatenate((A, B), axis=1)

# Concatenate rows of an array
def concat_rows(A, B):
    return np.concatenate((A, B), axis=0)

# Split an array into columns
def split_cols(A):
    mid = A.shape[1] // 2
    return A[:, :mid], A[:, mid:]

# Split an array into rows
def split_rows(A):
    mid = A.shape[0] // 2
    return A[:mid, :], A[mid:, :]


### Transform Functions

In [8]:
import numpy as np
import pywt
from scipy.fftpack import dct, idct

# Discrete Cosine Transform
def DCT(A):
    return dct(dct(A.T, norm='ortho').T, norm='ortho')

# Inverse Discrete Cosine Transform
def DCT_inv(A):
    return idct(idct(A.T, norm='ortho').T, norm='ortho')

def Haar(A):
    # Perform the Haar transform and convert coefficients to array
    coeffs = pywt.wavedecn(A, 'haar')
    arr, coeff_slices = pywt.coeffs_to_array(coeffs)
    return arr, coeff_slices

def Haar_inv(arr, coeff_slices):
    # Convert array back to coefficients and perform the inverse transform
    coeffs = pywt.array_to_coeffs(arr, coeff_slices, output_format='wavedecn')
    return pywt.waverecn(coeffs, 'haar')

def DCT1_Haar1(A):
    # Perform DCT and then Haar transform
    dct_transformed = DCT(A)
    haar_transformed, coeff_slices = Haar(dct_transformed)
    return haar_transformed, coeff_slices

def DCT1_Haar1_inv(A, coeff_slices):
    # Perform inverse Haar and then inverse DCT
    haar_inv_transformed = Haar_inv(A, coeff_slices)
    return DCT_inv(haar_inv_transformed)


### Compression and Decompression Functions

In [9]:
import zlib
import struct

# Compress a float array using zlib
def compress_float(array):
    flat_array = array.flatten()
    packed_data = pack(f'{len(flat_array)}f', *flat_array)
    compressed_data = zlib.compress(packed_data)
    return compressed_data

# Decompress a float array using zlib
def decompress_float(data):
    decompressed_data = zlib.decompress(data)
    array = np.array(unpack(f'{len(decompressed_data) // 4}f', decompressed_data), dtype=np.float32)
    return array


### Code and decode functions

In [10]:
import numpy as np
from PIL import Image
import zlib
from struct import pack, unpack

# Define the paths for input and output files
input_file = 'fif-codec/images/input/lena.png'
encoded_file = 'encoded_lena.fif'

def read_image(file_path):
    img = Image.open(file_path)
    img = img.convert('L')  # Convert to grayscale
    return np.array(img, dtype=np.float32)

def save_image(array, file_path):
    img = Image.fromarray(array.astype(np.uint8))
    img.save(file_path)

def code(img_file, encoded_file, A_name, max_error, bi, callback=None, ssim_stop=False, min_n=8, max_n=32):
    img = read_image(img_file)
    H, W = img.shape
    encoded_bytes = b""
    
    if A_name == "DCT":
        encoded_data = DCT(img)
        coeff_slices = None
    elif A_name == "Haar":
        encoded_data, coeff_slices = Haar(img)
    elif A_name == "DCT1_Haar1":
        encoded_data, coeff_slices = DCT1_Haar1(img)
    else:
        raise ValueError("Unsupported basis function name")
    
    compressed_data = compress_float(encoded_data)
    encoded_bytes += pack('II', H, W)
    
    # Store number of coeff_slices only if A_name is "DCT1_Haar1"
    if coeff_slices:
        encoded_bytes += pack('I', len(coeff_slices))  # Store number of slices
    encoded_bytes += compressed_data
    
    with open(encoded_file, 'wb') as f:
        f.write(encoded_bytes)
    return len(encoded_bytes)

def decode(encoded_file, output_file, A_name):
    with open(encoded_file, 'rb') as f:
        H, W = unpack('II', f.read(8))
        
        # Read number of coeff_slices only if A_name is "DCT1_Haar1"
        if A_name == "DCT1_Haar1":
            num_slices = unpack('I', f.read(4))[0]
            coeff_slices = [None] * num_slices
        else:
            coeff_slices = None
        
        compressed_data = f.read()
    
    encoded_data = decompress_float(compressed_data)
    
    if A_name == "DCT":
        decoded_img = DCT_inv(encoded_data)
    elif A_name == "Haar":
        decoded_img = Haar_inv(encoded_data, None)
    elif A_name == "DCT1_Haar1":
        decoded_img = DCT1_Haar1_inv(encoded_data, coeff_slices)
    else:
        raise ValueError("Unsupported basis function name")

    save_image(decoded_img, output_file)

### Running code and decode functions

In [11]:
# Define the paths for input and output files
input_file = '../images/lena.png'
encoded_file = 'encoded_lena.fif'
decoded_file = 'decoded_lena.jpg'

# Parameters for encoding
A_name = "DCT1_Haar1"
max_error = 1.0
bi = 1  # Assuming bijection index 1 (concat_cols)
callback = None
ssim_stop = False
min_n = 8
max_n = 32

# Encoding the image
encoded_result = code(input_file, encoded_file, A_name, max_error, bi, callback, ssim_stop, min_n, max_n)
print(f"Encoding complete. Results: {encoded_result}")

# Decoding the image
decode(encoded_file, decoded_file, A_name)  # Pass A_name here
print(f"Decoding complete. Output saved to {decoded_file}")

Encoding complete. Results: 975733


AttributeError: 'NoneType' object has no attribute 'items'

### Step by step code and decode functions

In [53]:
import numpy as np
from PIL import Image
import zlib
from struct import pack, unpack

# Define the paths for input and output files
input_file = 'fif-codec/images/input/lena.png'
encoded_file = 'encoded_lena.fif'

img = Image.open(input_file)
img = img.convert('L')  # Convert to grayscale
img_array=np.array(img, dtype=np.float32)

# Encoding and decoding should be done with each channel
# img_RGB = img.convert('RGB')
# img_array_RGB=np.array(img_RGB, dtype=np.float32)

print(img_array.shape)

(512, 512)


In [42]:
H, W = img_array.shape
print(img_array.shape)
encoded_bytes = b""

encoded_data, coeff_slices = DCT1_Haar1(img_array)
print(encoded_data)
print(coeff_slices)

(512, 512)
[[ 1.39747437e+02  1.38105499e+02  2.80572021e+02 ... -2.82519531e+00
   1.58691406e-01  1.76819146e-01]
 [ 1.39672424e+02  1.39361847e+02  1.18498302e+00 ... -7.64503539e-01
  -1.46705616e+00  6.67556763e-01]
 [ 2.75843353e+02  2.28394413e+00  2.81807037e+02 ... -7.03926086e-01
   2.70566940e+00  3.47739363e+00]
 ...
 [ 7.23073387e+00 -9.77244759e+00  6.70488596e-01 ...  1.05906606e-01
   4.09327269e-01  2.49979281e+00]
 [ 3.82510304e+00  2.59905577e+00 -5.37048197e+00 ... -4.42273974e-01
   7.09844649e-01 -4.82821584e-01]
 [ 1.17933714e+00  5.48841095e+00 -2.97481966e+00 ... -2.99732506e-01
   2.94415569e+00 -3.84998417e+00]]
[(slice(None, 1, None), slice(None, 1, None)), {'ad': (slice(None, 1, None), slice(1, 2, None)), 'da': (slice(1, 2, None), slice(None, 1, None)), 'dd': (slice(1, 2, None), slice(1, 2, None))}, {'ad': (slice(None, 2, None), slice(2, 4, None)), 'da': (slice(2, 4, None), slice(None, 2, None)), 'dd': (slice(2, 4, None), slice(2, 4, None))}, {'ad': (slice(

In [43]:
import pickle

# Assuming `coeff_slices` is obtained during the Haar transform
compressed_data = compress_float(encoded_data)
encoded_bytes = pack('II', H, W)
print(encoded_bytes)

if coeff_slices:
    serialized_slices = pickle.dumps(coeff_slices)
    encoded_bytes += pack('I', len(serialized_slices))
    print(encoded_bytes)
    encoded_bytes += serialized_slices
    print(encoded_bytes)
else:
    encoded_bytes += pack('I', 0)

encoded_bytes += compressed_data

with open(encoded_file, 'wb') as f:
    f.write(encoded_bytes)

b'\x00\x02\x00\x00\x00\x02\x00\x00'
b'\x00\x02\x00\x00\x00\x02\x00\x00h\x03\x00\x00'
b'\x00\x02\x00\x00\x00\x02\x00\x00h\x03\x00\x00\x80\x04\x95]\x03\x00\x00\x00\x00\x00\x00]\x94(\x8c\x08builtins\x94\x8c\x05slice\x94\x93\x94NK\x01N\x87\x94R\x94h\x03NK\x01N\x87\x94R\x94\x86\x94}\x94(\x8c\x02ad\x94h\x03NK\x01N\x87\x94R\x94h\x03K\x01K\x02N\x87\x94R\x94\x86\x94\x8c\x02da\x94h\x03K\x01K\x02N\x87\x94R\x94h\x03NK\x01N\x87\x94R\x94\x86\x94\x8c\x02dd\x94h\x03K\x01K\x02N\x87\x94R\x94h\x03K\x01K\x02N\x87\x94R\x94\x86\x94u}\x94(\x8c\x02ad\x94h\x03NK\x02N\x87\x94R\x94h\x03K\x02K\x04N\x87\x94R\x94\x86\x94\x8c\x02da\x94h\x03K\x02K\x04N\x87\x94R\x94h\x03NK\x02N\x87\x94R\x94\x86\x94\x8c\x02dd\x94h\x03K\x02K\x04N\x87\x94R\x94h\x03K\x02K\x04N\x87\x94R\x94\x86\x94u}\x94(\x8c\x02ad\x94h\x03NK\x04N\x87\x94R\x94h\x03K\x04K\x08N\x87\x94R\x94\x86\x94\x8c\x02da\x94h\x03K\x04K\x08N\x87\x94R\x94h\x03NK\x04N\x87\x94R\x94\x86\x94\x8c\x02dd\x94h\x03K\x04K\x08N\x87\x94R\x94h\x03K\x04K\x08N\x87\x94R\x94\x86\x94u}\x94(

In [44]:
with open(encoded_file, 'rb') as f:
    H, W = unpack('II', f.read(8))
    num_slices = unpack('I', f.read(4))[0]
    if num_slices > 0:
        serialized_slices = f.read(num_slices)
        loaded_DCT1_Haar1_coeff = pickle.loads(serialized_slices)
    else:
        loaded_DCT1_Haar1_coeff = None
    compressed_data = f.read()

encoded_data = decompress_float(compressed_data)
encoded_data = np.reshape(encoded_data, (H, W))
# decoded_img = DCT1_Haar1_inv(encoded_data, loaded_DCT1_Haar1_coeff)

In [55]:
encoded_data.shape, len(loaded_DCT1_Haar1_coeff)

((512, 512), 10)

In [46]:
decoded_img = DCT1_Haar1_inv(encoded_data, coeff_slices)

In [48]:
decoded_file = 'decoded_lena.jpg'
save_image(decoded_img, decoded_file)